### circuit
+ Synchronous circuit: governed by a global clock signal
+ Asynchronous circuit: change states only through the inputs received by them

### Gate

**NOR**
Inverted or gate, i.e., $Q = \neg{(A \lor B)}$
<img src="https://upload.wikimedia.org/wikipedia/commons/c/c6/NOR_ANSI_Labelled.svg" width=150/>

### Modules
#### Flip-flop
+ 2 stable states, controlled by control inputs
+ store a single bit


**types**
+ asynchronous => latch (level-sensitive, sequential elements meaning non-blocking assignments)
+ synchronous => flip-flop (edge-sensitive)


**D flip-flop.**
D flip-flop stores a bit and is updated periodically, at the (usually) positive edge of a clock signal. See details [here](https://hdlbits.01xz.net/wiki/Dff). D flip-flop uses non-blocking syntax. D flip-flops is a extension from D flip-flop with multiple-bits inputs and outputs, i.e., the only difference between D flip-flop and D flip-flops is that the latter has its input and output as vectors.

```verilog
// D flip-flop(s)
always @(posedge clk) begin
    q <= d;
end

// Example of D flip-flops with synchronous reset value
always @(posedge clk) begin
    if (reset) begin
        q <= 8'b0;
    end else begin
     	q <= d;
    end
end

// Example of D flip-flops with asynchronous reset value (2 key points: 1. posedge areset; 2. areset==1'b1)
always @ (posedge areset or posedge clk) begin
    if (areset==1'b1) begin
     	q <= 8'b0;
    end else begin
        q <= d;
    end
end
```


**D Latch.**
D latch acts like a wire when enabled, and preserves the current value when disabled. See details [here](https://hdlbits.01xz.net/wiki/Exams/m2014_q4a).
<img src="https://hdlbits.01xz.net/mw/images/0/03/Exams_m2014q4a.png" width="50"/>

```verilog
// D Latch
always @ (ena) begin
    if (ena) begin
        q <= d;
    end
end
```


**SR NOR Latch**

+ S: Set
+ R: Reset

Important:
+ $S=1, R=1$ => not allowed 
+ $S=0, R=0$ OR $S=1 \rightarrow 0, R=0$ OR $R=1 \rightarrow 0, S=0$ => Q remains unchanged
+ $S=1$ => $Q=1, \bar{Q}=0$
+ $R=1$ => $Q=0, \bar{Q}=1$

<img src="https://upload.wikimedia.org/wikipedia/commons/c/c6/R-S_mk2.gif" width=150/>

#### Multiplexer
A combinational logic circuit designed to switch one of several input lines to a single common output line.


For level sensitive 2:1 multiplexer (Sel means select input):
<img src="https://upload.wikimedia.org/wikipedia/commons/b/b2/Multiplexer2.png" width=350/>



### Verilog

Website for practicing Verilog: [HDLBits — Verilog Practice](https://hdlbits.01xz.net/wiki/Main_Page)

**number.** b, o, d, h

**modules**
Things with inputs, outputs and internal logic workings. A rough equivalents of functions with returns in other programming languages. 
+ Inputs and outputs can also be called *ports*. 
+ Bi-directional ports are defined as *inout*.
+ Our task is to create a connection to connect inputs to outputs.
+ Hierarchy: We can instantiate a module inside another. The module declaration

**Vector**
vector signal: `inout [7:0] address`(little-endian convention, i.e, start with 0 at the rightmost bit to begin the vector, while `[0:7]` represents big-endian. `[7:0]` creates 8-element vector). `assign out = address[7]` selects the 1 bit (i.e., 7th bit) out of the vector. 
+ Vector indices can be variable, as long as the synthesizer can figure out that the width of the bits being selected is constant. In particular, selecting one bit out of a vector using a variable index will work. For example, `assign out = in[sel]` is legal, where `in` is 256 bits and `sel` is 8 bits.
+ We can't part-select multiple bits without an error, but we can select one bit at a time, four times, then concatenate them together. For example, `assign out = in[sel * 4 + 3:sel * 4]` is incorrect, but `assign out = {in[sel*4+3], in[sel*4+2], in[sel*4+1], in[sel*4]}` is correct.

**Data Type**
no data types, but two kinds of drives in hardware world
+ wire: connect two points. directional. This means that information flows in only one direction, from *source* to the *sinks*. (The source is also often called a driver that drives a value onto a wire.)
    + `input wire a` (port_type data_type name) <=> `input a` (port_type name)
    + `assign` statement does not create a wire, but a connection between things(such as wires)
    + a wire cannot have more than one driver
    + a wire without driver has an undefined value
    + if the internal wire is exactly the output itself, we do not need to declare a new wire. See details [here](https://hdlbits.01xz.net/wiki/Wire_decl)
+ register: store a value (signed, unsigned, floating point, etc)


**Assignment**
+ Continuous assignment. Not a one-time event, i.e., the assignment continues even if the right side's value changes.
+ When there are multiple assignments, the order in which they appear in the code does not matter.
+ Describe connections between things, not the action of copying a value from one thing to another.

**Operation**
The operation in Verilog is similar as C. The operation can be applied to vectors as well.
+ Bitwise Operation: `~`(negation, inverse), `&`(and), `|`(or), `^`(xor), `^~`/`~^`(xnor)
+ Logical Operation: `&&`(logical and), `||`(logical or), `!`(logical not)
+ equality: For example, in `assign z = (A[1:0]==B[1:0])`, `z` produces 1 if and only if A and B are same, 0 otherwise. Note that `assign z = (A == B)` also makes sense.

**Numbers**
`x'[b|h]y` where `x` is the number of bits to store them, `b` or `h` means binary and hexadecimal prespectively and `y` is the exact value.
+ `1'b0`: binary 0
+ `16'hDEAD`: Hexadecimal
+ `'1`: is a special literal syntax for a number with all bits set to 1. `'0`, `'x`, and `'z` are also valid.

**Concatenation**
Concatenation operator `{a, b, c}` is used to create larger vectors by concatenating smaller portions of a vector togerther.
+ concatenation operator needs to know the number of bits, i.e., `{1, 2, 3}` is illegal
+ `{1'b1, 1'b0, 3'b101}` => `5'b10101`
+ concatenation operator can be used in both sides: `assign {out[7:0], out[15:8]} = in;` or `assign out[15:0] = {in[7:0], in[15:8]};`
+ We can reverse the vector using concatenation: `assign {out[0],out[1],out[2],out[3],out[4],out[5],out[6],out[7]} = in;`, where `in` and `out` are both 8-bit vectors. Another option is `assign out = {in[0], in[1], in[2], in[3], in[4], in[5], in[6], in[7]};`.
+ `{num{vector}}` can replicate concatenation. For example, `{5{1'b1}}` => `5'b11111`. Note that the nested replication is: `{{24{in[7]}}, in}`, instead of ~~`{24{in[7]}, in}`~~

**Initial Blocks**
Executed once at the beginning of simulation (i.e., when time = 0)
```verilog
initial begin
    clk = 0;
    reset = 0;
    req_0 = 0;
    req_1 = 0;
end
```

**always Blocks**
+ `@`: indicates that the condition is inside the parameters after `@`. The block is triggered "at" this condition.

cannot drive wire data, but can drive reg and integer data types

+ "always" means that whenever input variables changed, the codes are executed
+ sensitive list: 2 types
    + level sensitive: for combinational circuits(combinational circuits are time independent, i.e., not rely on clock cycles)
    + edge sensitive: for flip-flops


For level sensitive:
```verilog
always @ (a or b or sel)
begin
    y = 0;
    if (sel == 0); begin
        y = a;
    end else begin
        y = b;
    end
end
```

For edge sensitive:
```verilog
always @ (posedge clk)
if (reset == 0) begin
    y <= 0;
end else if (sel == 0) begin
    y <= a;
end else begin
    y <= b;
end
```
Every time when clock makes the transition from 0 to 1 (posedge), we check if reset is is asserted(synchronous reset), then we go no with normal logic.

+ `=`: blocking assignment. Executes codes sequentially inside a begin/end.
+ `<=`: nonblocking assignment. Execute codes in parallel.

However, we can have an always block without sensitive list:
```verilog
always begin
    #5 clk=~clk;
end
```
+ `#5`: delays execution by 5 time units



**case**
+ Case statement can only be used inside procedural blocks (always block)
+ This is a combinational circuit, so use a combinational `always @ (*)` block
```verilog
always @(*) begin
    case (sel)
        4'h0: out = a;
        4'h1: out = b;
        4'h2: out = c;
        4'h3: out = d;
        4'h4: out = e;
        4'h5: out = f;
        4'h6: out = g;
        4'h7: out = h;
        4'h8: out = i;
        default out = '1;
    endcase
end
```

### Coding Style
#### Declare a module
Verilog-1995 Syntax:
```verilog
module top_module(zero);
    output zero;
    // ...
endmodule
```
Verilog-2001 Syntax:
```verilog
module top_module( output zero );
    // ...
endmodule
```
#### Instantiate a module
Suppose a module is declared as follows:
```verilog
module mod_a ( input in1, input in2, output out );
    // Module body
endmodule
```
Then an instance of `mod_a` can be created as follows. We create a module and connect our existing wires to the ports of the new module. In the code below, `wa`, `wb` and `wc` are wires outside the module. The instance name is not necessary.
```verilog
mod_a instance1 ( wa, wb, wc ); // by position
//mod_a ( wa, wb, wc ); // by position
```
Or
```verilog
mod_a instance1 ( .in1(wa), .in2(wb), .out(wc) ); // by name
//mod_a ( .in1(wa), .in2(wb), .out(wc) ); // by name
```